In [1]:
%pylab inline
import PyDAQmx as dq

Populating the interactive namespace from numpy and matplotlib


In [2]:
# create tasks
# for EveryNCallback functionality create your own class, e.g.:
# class DigitalOutput(dq.Task)
DOTask = dq.Task()
AOTask = dq.Task()

In [3]:
# example pulses
# uint32 value maps to 32 lines:
# lsb -> Dev1/port0/line0 (odd number means 1, even means 0)
# ...
# msb -> Dev1/port0/line31 (2**31 and higher means 1)
DOData = np.array([2**15+100-x for x in range(0,1000)], dtype=np.uint32)
AOData = np.array([np.exp(-(x%100)/50) for x in range(0,200)], dtype=np.double)

In [4]:
# zero-pulses
DOData = np.array([0 for x in range(0,1000)], dtype=np.uint32)
AOData = np.array([0 for x in range(0,200)], dtype=np.double)

In [5]:
# create channels to write data to
DOTask.CreateDOChan('Dev1/port0',"",dq.DAQmx_Val_ChanForAllLines)
for x in range(0,2):
    AOTask.CreateAOVoltageChan('Dev1/ao'+str(x),"",-10.0,10.0,dq.DAQmx_Val_Volts,None)

In [6]:
# configure timing and sync
# DOTask uses /Dev1/do/SampleCLock
# AOTask uses /Dev1/ao/SampleClock
# AOTask triggers by /Dev1/do/StartTrigger
DOTask.CfgSampClkTiming("",1000000.0,dq.DAQmx_Val_Rising,dq.DAQmx_Val_ContSamps,1000)
AOTask.CfgSampClkTiming("",100000.0,dq.DAQmx_Val_Rising,dq.DAQmx_Val_ContSamps,100)
AOTask.CfgDigEdgeStartTrig('/Dev1/do/StartTrigger',dq.DAQmx_Val_Rising)

In [7]:
# write data
# remember to use dtype=np.uint8/double for np.array
#DOTask.WriteDigitalLines(1000,0,10.0,dq.DAQmx_Val_GroupByChannel,DOData,None,None)
DOTask.WriteDigitalU32(1000,0,10.0,dq.DAQmx_Val_GroupByChannel,DOData,None,None)
AOTask.WriteAnalogF64(100,0,10.0,dq.DAQmx_Val_GroupByChannel,AOData,None,None)

In [8]:
AOTask.StartTask()
DOTask.StartTask()

In [9]:
DOTask.StopTask()
AOTask.StopTask()

In [10]:
DOTask.ClearTask()
AOTask.ClearTask()

In [11]:
help(dq.Task.AutoRegisterEveryNSamplesEvent)

Help on function AutoRegisterEveryNSamplesEvent in module PyDAQmx.Task:

AutoRegisterEveryNSamplesEvent(self, everyNsamplesEventType, nSamples, options, name='EveryNCallback')
    Register the method named name as the callback function for EveryNSamplesEvent
    
    
    With this method you can register a method of the class Task as a callback function. 
    The parameters everyNsamplesEventType, nSamples and options are the same 
    as the DAQmxRegisterEveryNSamplesEvent parameters
    
    No parameters are passed to the method  
    
    If an event was already registered, the UnregisterEveryNSamplesEvent is automatically called

